# Example 1. Authorizing an application to access Twitter account data

In [1]:
# I had to add this because the ipython notebook could not find the Twitter library on my computer.
# You may not need to add these first two lines
import sys
sys.path.append('/home/matt/Enthought/Canopy_64bit/User/lib/python2.7/site-packages')
import twitter
# Go to http://dev.twitter.com/apps/new to create an app and get values
# for these credentials, which you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation.

# CONSUMER_KEY = 'Tx8GozuzaraJO89LypsTqFUNM'
# CONSUMER_SECRET ='uC0KPaRZjGY0qoNJAxBQb0jbb3Us4DXJVKFCP1qtSfjFgbRvd0'
# OAUTH_TOKEN = '571213367-BC7MYnWpLbvy2AYhvqCu6Fh1EIeFasPxo0Y8uBEn'
# OAUTH_TOKEN_SECRET = 'KA9GddiLbVwummLoSGmDYMNQFVaR3P4YiGo7Zs6D5e7Qz'

CONSUMER_KEY = 'YBicfvPPR7YmsJdFtd8qGATmI'
CONSUMER_SECRET ='YYXKSBw5n6b5dpbwmMfZTjqJcUnDQm6fxlNUEnf63k5idJsj2m'
OAUTH_TOKEN = '774076984561430528-1ENguVW2E40PSue6YWnDuCBUyASjhGB'
OAUTH_TOKEN_SECRET = 'ZcjrXboPxPmGEXzq2n1GQN1s7u8ZAXQeKWDSVIrZxQMkV'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

In [2]:
import json
import twitter

def twitter_trends(twitter_api, woe_id):
    # Prefix ID with the underscore for query string parameterization.
    # Without the underscore, the twitter package appends the ID value
    # to the URL itself as a special-case keyword argument.
    return twitter_api.trends.place(_id=woe_id)

# Sample usage

#twitter_api = oauth_login()

# See https://dev.twitter.com/docs/api/1.1/get/trends/place and
# http://developer.yahoo.com/geo/geoplanet/ for details on
# Yahoo! Where On Earth ID

WORLD_WOE_ID = 1
world_trends = twitter_trends(twitter_api, WORLD_WOE_ID)
#print (json.dumps(world_trends, indent=1))

US_WOE_ID = 23424977
us_trends = twitter_trends(twitter_api, US_WOE_ID)
#print (json.dumps(us_trends, indent=1))

In [3]:
def twitter_search(twitter_api, q, max_results=200, **kw):

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets and 
    # https://dev.twitter.com/docs/using-search for details on advanced 
    # search criteria that may be useful for keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://dev.twitter.com/docs/rate-limiting/1.1/limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

# Sample usage

#twitter_api = oauth_login()

q = "CrossFit"
results = twitter_search(twitter_api, q, max_results=10)
        
# Show one sample search result by slicing the list...
print (json.dumps(results[0], indent=1))

{
 "possibly_sensitive": false,
 "in_reply_to_user_id_str": null,
 "truncated": false,
 "created_at": "Tue Sep 20 01:35:38 +0000 2016",
 "in_reply_to_status_id_str": null,
 "favorite_count": 0,
 "entities": {
  "hashtags": [],
  "user_mentions": [
   {
    "id": 3068796514,
    "name": "BuffBoxx",
    "indices": [
     3,
     12
    ],
    "screen_name": "buffboxx",
    "id_str": "3068796514"
   }
  ],
  "urls": [
   {
    "url": "https://t.co/Wl1aDP0pxN",
    "expanded_url": "http://ow.ly/41W8504xFDy",
    "indices": [
     67,
     90
    ],
    "display_url": "ow.ly/41W8504xFDy"
   }
  ],
  "symbols": []
 },
 "in_reply_to_status_id": null,
 "id": 778044917532721152,
 "coordinates": null,
 "place": null,
 "user": {
  "default_profile": false,
  "is_translator": false,
  "profile_background_color": "131516",
  "created_at": "Sun Aug 22 00:40:37 +0000 2010",
  "description": "Fotografia & Video https://t.co/g4m8Vhy8Wp",
  "profile_sidebar_border_color": "000000",
  "follow_request_sen

In [4]:
from functools import partial

pp = partial(json.dumps, indent=1)

twitter_world_trends = partial(twitter_trends, twitter_api, WORLD_WOE_ID)

#print (pp(twitter_world_trends()))

authenticated_twitter_search = partial(twitter_search, twitter_api)
results = authenticated_twitter_search("iPhone")
#print (pp(results))

authenticated_iphone_twitter_search = partial(authenticated_twitter_search, "iPhone")
results = authenticated_iphone_twitter_search()
#print (pp(results))

In [5]:
import sys
import time
from urllib.error import URLError
from http.client import BadStatusLine
import json
import twitter

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw): 
    
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
    
        if wait_period > 3600: # Seconds
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
    
        # See https://dev.twitter.com/docs/error-codes-responses for common codes
    ''' if e.e.code == 401:
            print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
            return None
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        elif e.e.code == 429: 
            print >> sys.stderr, 'Encountered 429 Error (Rate Limit Exceeded)'
            if sleep_when_rate_limited:
                print >> sys.stderr, "Retrying in 15 minutes...ZzZ..."
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print >> sys.stderr, '...ZzZ...Awake now and trying again.'
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print >> sys.stderr, 'Encountered %i Error. Retrying in %i seconds' % \
                (e.e.code, wait_period)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e '''
        

    # End of nested helper function 
    
    wait_period = 2 
    error_count = 0 

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError:
            error_count = 0 
            wait_period = handle_twitter_http_error(wait_period)
            if wait_period is None:
                return
        except URLError:
            error_count += 1
            print >> sys.stderr, "URLError encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise
        except BadStatusLine:
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

# Sample usage

#twitter_api = oauth_login()

# See https://dev.twitter.com/docs/api/1.1/get/users/lookup for 
# twitter_api.users.lookup

response = make_twitter_request(twitter_api.users.lookup, 
                                screen_name="SocialWebMining")

#print (json.dumps(response, indent=1))

In [6]:
from functools import partial
#from sys import maxint

def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=100000, followers_limit=100000):
    
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    # See https://dev.twitter.com/docs/api/1.1/get/friends/ids and
    # https://dev.twitter.com/docs/api/1.1/get/followers/ids for details
    # on API parameters
    
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids, 
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids, 
                                count=5000)
    

    friends_ids, followers_ids = [], []
    
    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"], 
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:
        
        if limit == 0: continue
        
        cursor = -1
        while cursor != 0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
            #print('Fetched {0} total {1} ids for {2}'.format(len(ids), 
                                                    #label, (user_id or screen_name)), end="", file=sys.stderr)
            #print >> sys.stderr, 'Fetched {0} total {1} ids for {2}'.format(len(ids), 
                                                    #label, (user_id or screen_name))
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances
        
            if len(ids) >= limit or response is None:
                break

    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]

# Sample usage

#twitter_api = oauth_login()

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       screen_name="JimmyG_10", 
                                                       friends_limit=20, 
                                                       followers_limit=20)

print (friends_ids)
print (followers_ids)


[376986779, 237527660, 460658453, 210567673, 3998034981, 327555798, 290809659, 183487809, 3115775917, 316901934, 3020277803, 252266942, 329385055, 829673054, 294318508, 2286022274, 1140790656, 220408962, 353990363, 128102424]
[356431090, 501311935, 156047028, 482513187, 2856054773, 157587353, 2752198528, 56626487, 403146359, 13186912, 2881312943, 777543595167457280, 2960369602, 97781452, 2939724693, 3161650337, 2205768813, 3407958670, 778004773203169281, 99599151]


In [7]:
def setwise_friends_followers_analysis(screen_name, friends_ids, followers_ids):
    
    friends_ids, followers_ids = set(friends_ids), set(followers_ids)
    
    print ('{0} is following {1}'.format(screen_name, len(friends_ids)))

    print ('{0} is being followed by {1}'.format(screen_name, len(followers_ids)))
    
    print ('{0} of {1} are not following {2} back'.format(
            len(friends_ids.difference(followers_ids)), 
            len(friends_ids), screen_name))
    
    print ('{0} of {1} are not being followed back by {2}'.format(
            len(followers_ids.difference(friends_ids)), 
            len(followers_ids), screen_name))
    
    print ('{0} has {1} mutual friends'.format(
            screen_name, len(friends_ids.intersection(followers_ids))))

# Sample usage

screen_name = "JimmyG_10"

#twitter_api = oauth_login()

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       screen_name=screen_name)
setwise_friends_followers_analysis(screen_name, friends_ids, followers_ids)

JimmyG_10 is following 222
JimmyG_10 is being followed by 70000
201 of 222 are not following JimmyG_10 back
69979 of 70000 are not being followed back by JimmyG_10
JimmyG_10 has 21 mutual friends


In [8]:
def get_user_profile(twitter_api, screen_names=None, user_ids=None):
   
    # Must have either screen_name or user_id (logical xor)
    assert (screen_names != None) != (user_ids != None), \
    "Must have screen_names or user_ids, but not both"
    
    items_to_info = {}

    items = screen_names or user_ids
    
    while len(items) > 0:

        # Process 100 items at a time per the API specifications for /users/lookup.
        # See https://dev.twitter.com/docs/api/1.1/get/users/lookup for details.
        
        items_str = ','.join([str(item) for item in items[:100]])
        items = items[100:]

        if screen_names:
            response = make_twitter_request(twitter_api.users.lookup, 
                                            screen_name=items_str)
        else: # user_ids
            response = make_twitter_request(twitter_api.users.lookup, 
                                            user_id=items_str)
    
        for user_info in response:
            if screen_names:
                items_to_info[user_info['screen_name']] = user_info
            else: # user_ids
                items_to_info[user_info['id']] = user_info

    return items_to_info

# Sample usage

#twitter_api = oauth_login()

#print (get_user_profile(twitter_api, user_ids=[776252947449667584])) 
#print get_user_profile(twitter_api, user_ids=[132373965])

In [9]:
from collections import Counter

def harvest_user_timeline(twitter_api, screen_name=None, user_id=None, max_results=1000):
     
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"    
    
    kw = {  # Keyword args for the Twitter API call
        'count': 200,
        'trim_user': 'true',
        'include_rts' : 'true',
        'since_id' : 1
        }
    
    if screen_name:
        kw['screen_name'] = screen_name
    else:
        kw['user_id'] = user_id
        
    max_pages = 16
    results = []
    
    tweets = make_twitter_request(twitter_api.statuses.user_timeline, **kw)
    
    if tweets is None: # 401 (Not Authorized) - Need to bail out on loop entry
        tweets = []
        
    results += tweets
    
    #print >> sys.stderr, 'Fetched %i tweets' % len(tweets)
    
    page_num = 1
    
    # Many Twitter accounts have fewer than 200 tweets so you don't want to enter
    # the loop and waste a precious request if max_results = 200.
    
    # Note: Analogous optimizations could be applied inside the loop to try and 
    # save requests. e.g. Don't make a third request if you have 287 tweets out of 
    # a possible 400 tweets after your second request. Twitter does do some 
    # post-filtering on censored and deleted tweets out of batches of 'count', though,
    # so you can't strictly check for the number of results being 200. You might get
    # back 198, for example, and still have many more tweets to go. If you have the
    # total number of tweets for an account (by GET /users/lookup/), then you could 
    # simply use this value as a guide.
    
    if max_results == kw['count']:
        page_num = max_pages # Prevent loop entry
    
    while page_num < max_pages and len(tweets) > 0 and len(results) < max_results:
    
        # Necessary for traversing the timeline in Twitter's v1.1 API:
        # get the next query's max-id parameter to pass in.
        # See https://dev.twitter.com/docs/working-with-timelines.
        kw['max_id'] = min([ tweet['id'] for tweet in tweets]) - 1 
    
        tweets = make_twitter_request(twitter_api.statuses.user_timeline, **kw)
        results += tweets

        #print >> sys.stderr, 'Fetched %i tweets' % (len(tweets),)
    
        page_num += 1
        
    #print >> sys.stderr, 'Done fetching tweets'

    return results[:max_results]
    
# Sample usage

#twitter_api = oauth_login()

#tweets = []
important = []
for j in range(9):
    tweets=harvest_user_timeline(twitter_api, user_id=followers_ids[j], \
                                   max_results=40)
    if len(tweets)>5:
        unimportant =[]
        for i in range(5):
            unimportant.append(tweets[i]["text"])
        important.append(unimportant)
        
#print(important)

status_texts = important

#print(status_texts)

words=[]

for user in status_texts:
    userword=[]
    for tweet in user:
        for word in tweet.split():
            userword.append(word)
    for word in list(set(userword)):
        words.append(word)
        
        

#words = [ w 
    #      for t in status_texts 
        #      for w in list(Counter(t.split()).keys)
              #      ]

#for t in status_texts:
   # s = t.split():
    #for w in list(Counter(s).keys()):
        
        
#words = [ w 
          #for w in list(Counter(r).keys())]

print (words)


#tweet_entities = [d
          #        for d in words 
          #       ]
#print(tweet_entities)

print(Counter(words).most_common())


#print(tweets)

# Save to MongoDB with save_to_mongo or a local file with save_json...

['como', 'vayanse', 'que', 'es', 'algo', 'a', 'Hasta', 'tía', 'QB', 'le', 'Jimmy', 'comieron', '–Se', 'completo', 'escuchó', 'tu', 'Brady', 'vez', 'Jiménez', 'falta', 'para', 'retire', 'Helen?', 'indiscutible.', 'verga', 'Azcarate.', '😂😂😍😍', 'excelente', 'nombre', '#NoANarcoSeries', 'los', 'una', 'el', 'Tom', '–Y', 'titular', 'telenovela', 'Elba', 'la', 'sea', 'debe', 'Garoppolo', 'más...', 'ser', 'de', 'Televisa', 'mucho', 'pero', 'se', '@elbajimenez9', 'Hummels', '#SiANarcoSeries', 'eso', 'gatos', 'go', '@MagicJohnson', 'lunch', 'the', 'a', 'Semis', 'YOUR', 'have', '#GoAngels', 'CFP', 'https://t.co/eH3BW6fQrj', 'Ultimate', 'to', 'Man', 'attend', 'Halos', 'prevails', 'FIGHTING', 'Pizza', 'DAME', 'Northern', 'Rangers.', '#mmfridaysweeps', 'bowl!', 'see', 'pick', 'https://t.c…', 'on', 'NOTRE', 'ALA/LSU.', 'take', 'Gooo', 'About', 'for', 'and', 'at', 'https://…', 'Rules:', 'GO', 'time', '#NotreDameFootball', 'Trout', 'Catch', 'delivers!', '@jemelehill', 'in', 'Experience', 'game!', 'seei

In [17]:
screen_names=[]
for j in range(10):
    screen_names.append(get_user_profile(twitter_api, user_ids=[followers_ids[j]])) 
#print(screen_names)
screen_names_2=[]
for j in range(10):
    screen_names_2.append(screen_names[j][followers_ids[j]]["screen_name"])
print(screen_names_2)

['lindsey_prince', 'ALEJANDROAZC', 'Tremendous61', 'dannnielle_', 'PatMoon123', 'ArturoCamachoS', '_sophiapeterson', 'Tim_Carroll00', 'KingBGK', 'Calif3Girl']
